In [ ]:
import pandas as pd
import requests
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
options = Options()
options.add_argument("--disable-notifications")
options.add_argument("--disable-popup-blocking")
driver = webdriver.Chrome(options=options)

In [ ]:
standings_url = 'https://fbref.com/en/comps/12/La-Liga-Stats'

In [ ]:
data = requests.get(standings_url)

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0] #css selector to select table with class name stats_table

links = [l.get('href') for l in standings_table.find_all('a')]
links = [l for l in links if '/squads/' in l]  #only want the links with /squads/ in it
team_urls = [f"https://fbref.com{l}" for l in links] #create list of all the teams urls/links
team_urls

In [ ]:
all_players=[]

for team_url in team_urls:
    driver.get(team_url)    
    time.sleep(2)

    try:
        driver.find_element(By.CSS_SELECTOR, 'button[mode=primary]').click()
    except Exception:
        pass

    time.sleep(2)
    togglep90 =  WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[id*='stats_defense_']")))
    tablename = driver.find_element(By.ID, "all_stats_defense")
    driver.execute_script("arguments[0].scrollIntoView();", tablename)

    time.sleep(2)
    togglep90.click()

    page_source = BeautifulSoup(driver.page_source, 'html.parser')
    tables = page_source.findAll('table', id=lambda x: x and x.startswith('stats_defense'))[0]
    defensivePlayerActions = pd.read_html(str(tables))[0]
    all_players.append(defensivePlayerActions)

driver.quit()


In [ ]:
df = pd.concat(all_players)

In [ ]:
df.columns = ['_'.join(col).strip() for col in df.columns.values]

In [ ]:
df = df.drop(["Challenges_Tkl", "Challenges_Att", "Challenges_Tkl%", "Challenges_Lost", "Unnamed: 21_level_0_Matches"], axis=1)

In [ ]:
df

In [ ]:
df = df.set_index("Unnamed: 0_level_0_Player")
df = df.drop(["Squad Total", "Opponent Total"])
df = df.reset_index()
df


In [ ]:
df = df[df["Unnamed: 2_level_0_Pos"]== "DF"]
df = df.reset_index()
df = df.drop("index", axis=1)
df

In [ ]:
df.columns = ['Player', 'Nation', 'Position', 'Age', '90s', 'Tackles', 'TacklesWon', 'TacklesDef3rd',
'TacklesMid3rd', 'TacklesAtt3rd', 'Blocks', 'ShotsBlocked', 'PassesBlocked', 'Intercptions', 'Tkl+Int', 'Clearances', 'Errors']

In [ ]:
df

In [ ]:
df.to_csv("LaLigadefenderP90.csv")